<a href="https://colab.research.google.com/github/RihaChri/PureNumpyConvNet/blob/main/PureNumpyConvNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import h5py
import matplotlib.pyplot as plt

%matplotlib inline
plt.rcParams['figure.figsize'] = (5.0, 4.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'
%load_ext autoreload
%autoreload 2
np.random.seed(1)

def zero_pad(X, pad):
    #X -- python numpy array of shape (m, n_H, n_W, n_C) representing a batch of m images
    #pad -- integer, amount of padding around each image on vertical and horizontal dimensions
    #X_pad -- padded image of shape (m, n_H + 2*pad, n_W + 2*pad, n_C)
    X_pad = np.pad(X,((0,0),(pad,pad),(pad,pad),(0,0)),'constant',constant_values = 0)
    return X_pad

def conv_single_step(a_slice_prev, W, b):
    #a_slice_prev -- slice of input data of shape (f, f, n_C_prev)
    #W -- Weight parameters contained in a window - matrix of shape (f, f, n_C_prev)
    #b -- Bias parameters contained in a window - matrix of shape (1, 1, 1)
    #Z -- a scalar value, the result of convolving the sliding window (W, b) on a slice x of the input data
    s = a_slice_prev * W
    Z = np.sum(s)
    Z = Z+b
    return Z

def conv_forward(A_prev, W, b, hparameters):
    #A_prev -- output activations of the previous layer, 
    #    numpy array of shape (m, n_H_prev, n_W_prev, n_C_prev)
    #W -- Weights, numpy array of shape (f, f, n_C_prev, n_C)
    #b -- Biases, numpy array of shape (1, 1, 1, n_C)
    #hparameters -- python dictionary containing "stride" and "pad"
    #Z -- conv output, numpy array of shape (m, n_H, n_W, n_C)
    #cache -- cache of values needed for the conv_backward() function
    (m, n_H_prev, n_W_prev, n_C_prev) = A_prev.shape
    (f, f, n_C_prev, n_C) = W.shape
    stride = hparameters['stride']
    pad = hparameters['pad']
    n_H = int((n_H_prev-f+2*pad)/stride)+1
    n_W = int((n_W_prev-f+2*pad)/stride)+1
    Z = np.zeros((m,n_H,n_W,n_C))
    A_prev_pad = zero_pad(A_prev,pad)
    
    for i in range(m):                              # loop over the batch of training examples
        a_prev_pad = A_prev_pad[i]                # Select ith training example's padded activation
        for h in range(n_H):                          # loop over vertical axis of the output volume
            for w in range(n_W):                      # loop over horizontal axis of the output volume
                for c in range(n_C):                  # loop over channels (= #filters) of the output volume
                    
                    # Find the corners of the current "slice" (≈4 lines)
                    # Note: there is a stride, therefore it is not wise to assign vert_start a value of h and it is the same with horiz_start
                    vert_start = h * stride
                    vert_end = vert_start + f
                    horiz_start = w * stride
                    horiz_end = horiz_start + f
                    
                    # Use the corners to define the (3D) slice of a_prev_pad (See Hint above the cell). (≈1 line)
                    a_slice_prev = a_prev_pad[vert_start:vert_end, horiz_start:horiz_end, :]
      
                    # Convolve the (3D) slice with the correct filter W and bias b, to get back one output neuron. (≈1 line)
                    Z[i, h, w, c] = np.sum(a_slice_prev * W[:, :, :, c]) + float(b[:, :, :, c])
                    Z[i, h, w, c] = conv_single_step(a_slice_prev, W[:, :, :, c], b[:, :, :, c])

    
    # Making sure your output shape is correct
    assert(Z.shape == (m, n_H, n_W, n_C))
    
    # Save information in "cache" for the backprop
    cache = (A_prev, W, b, hparameters)
    
    return Z, cache


def pool_forward(A_prev, hparameters, mode = "max"):
    #A_prev -- Input data, numpy array of shape (m, n_H_prev, n_W_prev, n_C_prev)
    #hparameters -- python dictionary containing "f" and "stride"
    #mode -- the pooling mode you would like to use, defined as a string ("max" or "average")
    #A -- output of the pool layer, a numpy array of shape (m, n_H, n_W, n_C)
    #cache -- cache used in the backward pass of the pooling layer, contains the input and hparameters 
    (m, n_H_prev, n_W_prev, n_C_prev) = A_prev.shape
    f = hparameters["f"]
    stride = hparameters["stride"]
    n_H = int(1 + (n_H_prev - f) / stride)
    n_W = int(1 + (n_W_prev - f) / stride)
    n_C = n_C_prev
    A = np.zeros((m, n_H, n_W, n_C))              
    
    for i in range(m):                          # loop over the training examples
        for h in range(n_H):                    # loop on the vertical axis of the output volume
            for w in range(n_W):                # loop on the horizontal axis of the output volume
                for c in range (n_C):           # loop over the channels of the output volume
                    
                    # Find the corners of the current "slice" (≈4 lines)
                    vert_start = h*stride
                    vert_end = vert_start+f
                    horiz_start = w*stride
                    horiz_end = horiz_start+f
                    
                    # Use the corners to define the current slice on the ith training example of A_prev, channel c. (≈1 line)
                    a_prev_slice = A_prev[i, vert_start:vert_end, horiz_start:horiz_end, c]
                    
                    # Compute the pooling operation on the slice. Use an if statment to differentiate the modes. Use np.max/np.mean.
                    if mode == "max":
                        A[i, h, w, c] = np.max(a_prev_slice)
                    elif mode == "average":
                        A[i, h, w, c] = np.sum(a_prev_slice)/(f*f)
    
    # Store the input and hparameters in "cache" for pool_backward()
    cache = (A_prev, hparameters)
    # Making sure your output shape is correct
    assert(A.shape == (m, n_H, n_W, n_C))
    return A, cache



def conv_backward(dZ, cache):
    #dZ -- gradient of the cost with respect to the output of the conv layer (Z), numpy array of shape (m, n_H, n_W, n_C)
    #cache -- cache of values needed for the conv_backward(), output of conv_forward()
    #dA_prev -- gradient of the cost with respect to the input of the conv layer (A_prev),
    #           numpy array of shape (m, n_H_prev, n_W_prev, n_C_prev)
    #dW -- gradient of the cost with respect to the weights of the conv layer (W)
    #      numpy array of shape (f, f, n_C_prev, n_C)
    #db -- gradient of the cost with respect to the biases of the conv layer (b)
    #      numpy array of shape (1, 1, 1, n_C)

    (A_prev, W, b, hparameters) = cache
    (m, n_H_prev, n_W_prev, n_C_prev) = A_prev.shape
    (f, f, n_C_prev, n_C) = W.shape
    stride = hparameters['stride']
    pad = hparameters['pad']
    (m, n_H, n_W, n_C) = dZ.shape
    dA_prev = np.zeros((m, n_H_prev, n_W_prev, n_C_prev))                           
    dW = np.zeros((f, f, n_C_prev, n_C))
    db = np.zeros((1, 1, 1, n_C))
    A_prev_pad = zero_pad(A_prev, pad)
    dA_prev_pad = zero_pad(dA_prev, pad)
    
    for i in range(m):                      # loop over the training examples
        
        # select ith training example from A_prev_pad and dA_prev_pad
        a_prev_pad = A_prev_pad[i, :]
        da_prev_pad = dA_prev_pad[i, :]
        
        for h in range(n_H):                   # loop over vertical axis of the output volume
            for w in range(n_W):               # loop over horizontal axis of the output volume
                for c in range(n_C):           # loop over the channels of the output volume
                    
                    # Find the corners of the current "slice"
                    vert_start = h*stride
                    vert_end = vert_start+f
                    horiz_start = w*stride
                    horiz_end = horiz_start+f
                    
                    # Use the corners to define the slice from a_prev_pad
                    a_slice = a_prev_pad[vert_start:vert_end, horiz_start:horiz_end, :]

                    # Update gradients for the window and the filter's parameters using the code formulas given above
                    da_prev_pad[vert_start:vert_end, horiz_start:horiz_end, :] += W[:,:,:,c] * dZ[i, h, w, c]
                    dW[:,:,:,c] += a_slice * dZ[i, h, w, c]
                    db[:,:,:,c] += dZ[i, h, w, c]
                    
        # Set the ith training example's dA_prev to the unpaded da_prev_pad (Hint: use X[pad:-pad, pad:-pad, :])
        dA_prev[i, :, :, :] = dA_prev_pad[i, pad:-pad, pad:-pad, :]
    
    # Making sure your output shape is correct
    assert(dA_prev.shape == (m, n_H_prev, n_W_prev, n_C_prev))
    
    return dA_prev, dW, db

def create_mask_from_window(x):
    #x -- Array of shape (f, f)
    #mask -- Array of the same shape as window, contains a True at the position corresponding to the max entry of x.
    mask = (x == np.max(x))
    
    return mask


def distribute_value(dz, shape):
    #dz -- input scalar
    #shape -- the shape (n_H, n_W) of the output matrix for which we want to distribute the value of dz
    #a -- Array of size (n_H, n_W) for which we distributed the value of dz
    (n_H, n_W) = shape
    average = n_H * n_W
    a = np.ones(shape) * dz / average
    
    return a


def pool_backward(dA, cache, mode = "max"):
    #dA -- gradient of cost with respect to the output of the pooling layer, same shape as A
    #cache -- cache output from the forward pass of the pooling layer, contains the layer's input and hparameters 
    #mode -- the pooling mode you would like to use, defined as a string ("max" or "average")
    #dA_prev -- gradient of cost with respect to the input of the pooling layer, same shape as A_prev
    (A_prev, hparameters) = cache
    stride = hparameters["stride"]
    f = hparameters["f"]
    m, n_H_prev, n_W_prev, n_C_prev = A_prev.shape
    m, n_H, n_W, n_C = dA.shape
    dA_prev = np.zeros(A_prev.shape)
    
    for i in range(m):                       # loop over the training examples
        #select training example from A_prev (≈1 line)
        a_prev = A_prev[i,:]
        for h in range(n_H):                 #  loop on the vertical axis
            for w in range(n_W):             #  loop on the horizontal axis
                for c in range(n_C):         #  loop over the channels (depth)
                    
                    # Find the corners of the current "slice" (≈4 lines)
                    vert_start = h * stride
                    vert_end = vert_start + f
                    horiz_start = w * stride
                    horiz_end = horiz_start + f 
                    
                    # Compute the backward propagation in both modes.
                    if mode == "max":
                        # Use the corners and "c" to define the current slice from a_prev (≈1 line)
                        a_prev_slice = a_prev[vert_start:vert_end,horiz_start:horiz_end,c]
                        # Create the mask from a_prev_slice (≈1 line)
                        mask = create_mask_from_window(a_prev_slice)
                        # Set dA_prev to be dA_prev + (the mask multiplied by the correct entry of dA) (≈1 line)
                        dA_prev[i, vert_start: vert_end, horiz_start: horiz_end, c] += mask * dA[i,h,w,c]
                        
                    elif mode == "average":
                        
                        # Get the value a from dA (≈1 line)
                        da = dA[i, h, w , c]
                        # Define the shape of the filter as fxf (≈1 line)
                        shape = (f,f)
                        # Distribute it to get the correct slice of dA_prev. i.e. Add the distributed value of da. (≈1 line)
                        dA_prev[i, vert_start: vert_end, horiz_start: horiz_end, c] += distribute_value(da, shape)
                        
  
    # Making sure your output shape is correct
    assert(dA_prev.shape == A_prev.shape)
    
    return dA_prev



np.random.seed(1)
A_prev = np.random.randn(5, 5, 3, 2)
hparameters = {"stride" : 1, "f": 2}
A, cache = pool_forward(A_prev, hparameters)
dA = np.random.randn(5, 4, 2, 2)

dA_prev = pool_backward(dA, cache, mode = "max")
print("mode = max")
print('mean of dA = ', np.mean(dA))
print('dA_prev[1,1] = ', dA_prev[1,1])  
print()
dA_prev = pool_backward(dA, cache, mode = "average")
print("mode = average")
print('mean of dA = ', np.mean(dA))
print('dA_prev[1,1] = ', dA_prev[1,1]) 

mode = max
mean of dA =  0.14571390272918056
dA_prev[1,1] =  [[ 0.          0.        ]
 [ 5.05844394 -1.68282702]
 [ 0.          0.        ]]

mode = average
mean of dA =  0.14571390272918056
dA_prev[1,1] =  [[ 0.08485462  0.2787552 ]
 [ 1.26461098 -0.25749373]
 [ 1.17975636 -0.53624893]]
